## Metoda eliminacji Gaussa

Rozwiązanie układu równań liniowych metodą eliminacji Gaussa polega na przekształceniu macierzy współczynników układu równań na macierz trójkątną górną, a następnie na rozwiązaniu układu równań za pomocą metody podstawiania wstecz.

Jest to jedna z najbardziej podstawowych metod rozwiązywania układów równań liniowych, wychodząca poza sytuacje gdzie liczba niewiadomych jest różna od liczby równań.

In [1]:
from sympy import Matrix, symbols, Eq, latex
from IPython.display import display, Markdown
import sympy as sp

class SymbolicMatrix:
    def __init__(self, matrix, column_labels=None, free_terms=None, display_mode="table"):
        self.matrix = Matrix(matrix)
        self.operations = []
        self.display_mode = display_mode

        if column_labels:
            if len(column_labels) != self.matrix.cols:
                raise ValueError("Liczba etykiet kolumn musi być równa liczbie kolumn w macierzy.")
            self.column_labels = column_labels
        else:
            self.column_labels = [f"x{i+1}" for i in range(self.matrix.cols)]

        if free_terms:
            if len(free_terms) != self.matrix.rows:
                raise ValueError("Liczba wyrazów wolnych musi być równa liczbie wierszy w macierzy.")
            self.free_terms = free_terms
        else:
            self.free_terms = [0 for _ in range(self.matrix.rows)]

        display(Markdown("**Początkowe równania:**"))
        self.display_matrix()

    def set_display_mode(self, mode):
        if mode not in ["table", "equations"]:
            raise ValueError("Tryb wyświetlania musi być 'table' lub 'equations'.")
        self.display_mode = mode

    def display_matrix(self):
        if self.display_mode == "table":
            self._display_table()
        elif self.display_mode == "equations":
            self._display_equations()

    def _display_table(self):
        matrix_with_free_terms = self.matrix.row_join(Matrix(self.free_terms))
        headers = self.column_labels + ["Wyraz wolny"]
        display(Markdown(self._matrix_to_markdown(matrix_with_free_terms, headers)))

    def _matrix_to_markdown(self, matrix, headers):
        markdown_str = "| " + " | ".join(headers) + " |\n"
        markdown_str += "| " + " | ".join(["---"] * len(headers)) + " |\n"
        for i in range(matrix.rows):
            row = [str(matrix[i, j]) for j in range(matrix.cols)]
            markdown_str += "| " + " | ".join(row) + " |\n"
        return markdown_str

    def _display_equations(self):
        equations = []
        for i in range(self.matrix.rows):
            terms = []
            for j in range(self.matrix.cols):
                coef = self.matrix[i, j]
                var = symbols(self.column_labels[j])

                # Sprawdzamy, czy współczynnik jest dodatni, ujemny czy zero
                if coef > 0:
                    if j == 0:
                        terms.append(f"{latex(coef)} {latex(var)}")
                    else:
                        terms.append(f"+ {latex(coef)} {latex(var)}")
                elif coef < 0:
                    terms.append(f"- {latex(abs(coef))} {latex(var)}")
                elif coef == 0:
                    if j == 0:
                        terms.append(f"0 {latex(var)}")
                    else:
                        terms.append(f"+0 {latex(var)}")

            lhs = " ".join(terms)
            rhs = latex(self.free_terms[i])

            equation = f"{lhs} = {rhs}"
            equations.append(equation)

        display(Markdown("Układ równań:"))
        display(Markdown(f"$$\\begin{{aligned}} {self._equations_to_latex(equations)} \\end{{aligned}}$$"))

    def _equations_to_latex(self, equations):
        return " \\\\ ".join(equations)

    def add_row(self, target_row, source_row, coefficient):
        target_idx = self._validate_row_number(target_row)
        source_idx = self._validate_row_number(source_row)

        # Operacja na macierzy
        self.matrix.row_op(target_idx, lambda v, j: v + coefficient * self.matrix[source_idx, j])

        # Operacja na wyrazach wolnych
        self.free_terms[target_idx] += coefficient * self.free_terms[source_idx]

        operation_str = f"w{target_row} = w{target_row} + {coefficient}*w{source_row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def multiply_row(self, row, coefficient):
        row_idx = self._validate_row_number(row)

        # Operacja na macierzy
        self.matrix.row_op(row_idx, lambda v, _: coefficient * v)

        # Operacja na wyrazach wolnych
        self.free_terms[row_idx] *= coefficient

        operation_str = f"w{row} = {coefficient}*w{row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def swap_rows(self, row1, row2):
        row1_idx = self._validate_row_number(row1)
        row2_idx = self._validate_row_number(row2)

        self.matrix.row_swap(row1_idx, row2_idx)
        self.free_terms[row1_idx], self.free_terms[row2_idx] = self.free_terms[row2_idx], self.free_terms[row1_idx]

        operation_str = f"Zamiana w{row1} <-> w{row2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def swap_columns(self, col1, col2):
        col1_idx = self._validate_col_number(col1)
        col2_idx = self._validate_col_number(col2)

        self.matrix.col_swap(col1_idx, col2_idx)
        self.column_labels[col1_idx], self.column_labels[col2_idx] = self.column_labels[col2_idx], self.column_labels[col1_idx]

        operation_str = f"Zamiana kol{col1} <-> kol{col2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operacja:** {operation_str}"))
        self.display_matrix()

    def _validate_row_number(self, row):
        if not isinstance(row, int):
            raise TypeError("Numer wiersza musi być liczbą całkowitą.")
        if row < 1 or row > self.matrix.rows:
            raise IndexError(f"Numer wiersza musi być w zakresie od 1 do {self.matrix.rows}.")
        return row - 1

    def _validate_col_number(self, col):
        if not isinstance(col, int):
            raise TypeError("Numer kolumny musi być liczbą całkowitą.")
        if col < 1 or col > self.matrix.cols:
            raise IndexError(f"Numer kolumny musi być w zakresie od 1 do {self.matrix.cols}.")
        return col - 1

    def print_operations(self):
        display(Markdown("**Wykonane operacje:**"))
        for op in self.operations:
            print(op)

# Przykład użycia:
m = SymbolicMatrix([[1,2,3], [4,5,6], [7,8,12]], column_labels=["x1", "x2", "x3"], free_terms=[10, 11, 18], display_mode="equations")


**Początkowe równania:**

Układ równań:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 4 x_{1} + 5 x_{2} + 6 x_{3} = 11 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [2]:
m.add_row(2, 1, -4)

**Operacja:** w2 = w2 + -4*w1

Układ równań:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [3]:
m.add_row(3, 1, -7)

**Operacja:** w3 = w3 + -7*w1

Układ równań:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} - 6 x_{2} - 9 x_{3} = -52 \end{aligned}$$

In [4]:
m.add_row(3, 2, -2)

**Operacja:** w3 = w3 + -2*w2

Układ równań:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} +0 x_{2} + 3 x_{3} = 6 \end{aligned}$$

In [5]:
# mamy 3 równania, więc musimy zakończyć działanie
# definicja symboli
x1, x2, x3 = symbols('x1 x2 x3')
# Definicja równań
eq1 = Eq(x1 + 2*x2 + 3*x3, 10)
eq2 = Eq(-3*x2-6*x3,-29)
eq3 = Eq(3*x3, 6)

In [6]:
sol_x3 = sp.solve(eq3, x3)[0]
sol_x3

2

In [7]:
# podstawiamy rozwiązanie x3 do równania 2
eq2 = eq2.subs(x3, sol_x3)
# rozwiązujemy równanie 2
sol_x2 = sp.solve(eq2, x2)[0]
sol_x2

17/3

In [8]:
# podstawiamy rozwiązania x2 i x3 do równania 1
eq1 = eq1.subs({x2: sol_x2, x3: sol_x3})
# rozwiązujemy równanie 1
sol_x1 = sp.solve(eq1, x1)[0]
sol_x1

-22/3

---

## Zadania dla studentów

$$
\left\{\begin{array}{c}{x+2 y-2 z=4} \\ {2 x+y+z=0} \\ {3 x+2 y+z=1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y+z-t=2} \\ {2 x+y+z=3} \\ {-x+z-t=0} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y-z-t=0} \\ {2 x+3 y-2 z+t=4} \\ {3 x+5 z=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.
$$

In [24]:
m = SymbolicMatrix([[1,2,-2], [2,1,1], [3,2,1]], column_labels=["x", "y", "z"], free_terms=[4, 0, 1], display_mode="equations")

m.add_row(2, 1, -2)
m.add_row(3, 1, -3)
m.add_row(3, 2, -1)
m.add_row(1, 3, 2)
m.add_row(2, 3, -3)

x , y, z = symbols('x y z')
eq1 = Eq(x + 2*z,-2)
eq2 = Eq(-1*z, 1)
eq3 = Eq(-1*y + 2*z, -3)

sol_z = sp.solve(eq2, z)[0]
sol_z #-1
eq1 = eq1.subs(z, sol_z)
sol_x = sp.solve(eq1, x)[0]
sol_x # 0
eq3 = eq3.subs(z, sol_z)
sol_y = sp.solve(eq3, y)[0]
sol_y #1
print("x:",sol_x, "y:",sol_y, "z:",sol_z)

**Początkowe równania:**

Układ równań:

$$\begin{aligned} 1 x + 2 y - 2 z = 4 \\ 2 x + 1 y + 1 z = 0 \\ 3 x + 2 y + 1 z = 1 \end{aligned}$$

**Operacja:** w2 = w2 + -2*w1

Układ równań:

...

$$\begin{aligned} 1 x +0 y + 2 z = -2 \\ 0 x - 3 y + 5 z = -8 \\ 0 x - 1 y + 2 z = -3 \end{aligned}$$

**Operacja:** w2 = w2 + -3*w3

Układ równań:

$$\begin{aligned} 1 x +0 y + 2 z = -2 \\ 0 x +0 y - 1 z = 1 \\ 0 x - 1 y + 2 z = -3 \end{aligned}$$

x: 0 y: 1 z: -1


In [76]:
m = SymbolicMatrix([[1,1,1,-1], [2,1,1,0], [-1,0,1,-1],[3,2,-1,2]], column_labels=["x", "y", "z","t"], free_terms=[2, 3, 0,-1], display_mode="equations")

m.add_row(1,2,-1)
m.add_row(4,2,-2)
m.add_row(3,1,-1)
m.add_row(4,1,2)
m.add_row(1,1,-2)




x , y, z, t = symbols('x y z t')

eq1 = Eq(x+t,1)
eq2 = Eq(2*x + y + z, 3)
eq3 = Eq(z,1)
eq4 = Eq(-3*x-3*z,-9)

sol_z = sp.solve(eq3, z)[0]
sol_z #1
eq4 = eq4.subs(z, sol_z)
sol_x = sp.solve(eq4, x)[0]
sol_x #2
eq2 = eq2.subs({x: sol_x, z: sol_z})
sol_y = sp.solve(eq2, y)[0]
sol_y#-2
eq1 = eq1.subs(x,sol_x)
sol_t = sp.solve(eq1, t)[0]
sol_t #-1
print("x:",sol_x, "y:",sol_y, "z:",sol_z, "t:",sol_t)

**Początkowe równania:**

Układ równań:

$$\begin{aligned} 1 x + 1 y + 1 z - 1 t = 2 \\ 2 x + 1 y + 1 z +0 t = 3 \\ - 1 x +0 y + 1 z - 1 t = 0 \\ 3 x + 2 y - 1 z + 2 t = -1 \end{aligned}$$

**Operacja:** w1 = w1 + -1*w2

Układ równań:

...

$$\begin{aligned} - 1 x +0 y +0 z - 1 t = -1 \\ 2 x + 1 y + 1 z +0 t = 3 \\ 0 x +0 y + 1 z +0 t = 1 \\ - 3 x +0 y - 3 z +0 t = -9 \end{aligned}$$

**Operacja:** w1 = w1 + -2*w1

Układ równań:

$$\begin{aligned} 1 x +0 y +0 z + 1 t = 1 \\ 2 x + 1 y + 1 z +0 t = 3 \\ 0 x +0 y + 1 z +0 t = 1 \\ - 3 x +0 y - 3 z +0 t = -9 \end{aligned}$$

x: 2 y: -2 z: 1 t: -1


In [95]:
m = SymbolicMatrix([[1,1,-1,-1], [2,3,-2,1], [3,0,5,0],[-1,1,-3,2]], column_labels=["x", "y", "z","t"], free_terms=[0, 4, 0,3], display_mode="equations")

m.add_row(2,1,-2)
m.add_row(3,1,-3)
m.add_row(4,1,1)
m.add_row(1,2,-1)
m.add_row(3,2,3)
m.add_row(4,2,-2)

**Początkowe równania:**

Układ równań:

$$\begin{aligned} 1 x + 1 y - 1 z - 1 t = 0 \\ 2 x + 3 y - 2 z + 1 t = 4 \\ 3 x +0 y + 5 z +0 t = 0 \\ - 1 x + 1 y - 3 z + 2 t = 3 \end{aligned}$$

**Operacja:** w2 = w2 + -2*w1

Układ równań:

...

Układ równań:

$$\begin{aligned} 1 x +0 y - 1 z - 4 t = -4 \\ 0 x + 1 y +0 z + 3 t = 4 \\ 0 x +0 y + 8 z + 12 t = 12 \\ 0 x + 2 y - 4 z + 1 t = 3 \end{aligned}$$

**Operacja:** w4 = w4 + -2*w2

Układ równań:

$$\begin{aligned} 1 x +0 y - 1 z - 4 t = -4 \\ 0 x + 1 y +0 z + 3 t = 4 \\ 0 x +0 y + 8 z + 12 t = 12 \\ 0 x +0 y - 4 z - 5 t = -5 \end{aligned}$$